In [1]:
import utils

model_type = "3dcnn"
# model_type = "rnn"
model = utils.get_model(model_type = model_type, num_classes = 2)
model.eval()

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [4]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path2weights = "./models/weights_"+model_type+".pt"
 
model.load_state_dict(torch.load(path2weights))
model.to(device)

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [5]:
import os

path2data = "data"
sub_folder_jpg = "test_jpg_crop" 
path2ajpgs = os.path.join(path2data, sub_folder_jpg)

In [6]:
listOfImgs = os.listdir(path2ajpgs)
ids = []
for imgFolder in listOfImgs:
    ids.append(os.path.join(path2ajpgs, imgFolder))

In [7]:
ids[:3]

['data\\test_jpg_crop\\100032',
 'data\\test_jpg_crop\\100037',
 'data\\test_jpg_crop\\100139']

In [8]:
def make_prediction(imgs_tensor):
    with torch.no_grad():
        out = model(imgs_tensor.to(device)).cpu()
        pred = torch.argmax(out).item()
    return pred

In [9]:
preds = []
for PATH in ids:
    frames = utils.image_to_numpy(PATH)
    imgs_tensor = utils.transform_frames(frames, model_type="3dcnn")
    pred = make_prediction(imgs_tensor)
    preds.append(pred)

In [10]:
preds[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [11]:
import pandas as pd
df = pd.read_csv('submission_format.csv')
df = df[["filename"]]
df["stalled"] = preds
df.head()

,filename,stalled
0,100032.mp4,0
1,100037.mp4,0
2,100139.mp4,0
3,100182.mp4,0
4,100214.mp4,0


In [12]:
df.stalled.value_counts()

0    13143
1     1017
Name: stalled, dtype: int64

In [13]:
df.to_csv('submission_format4.csv', index=False)